In [23]:
import pandas as pd
import numpy as np

In [24]:
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fuzzywuzzy import process, fuzz
import pickle

import warnings
warnings.filterwarnings('ignore')

In [25]:
with open("datos/libros_scrap.pkl", "rb") as alias:
    libro2 =pickle.load(alias)

In [26]:
df_movies = pd.read_csv("imdb_top_1000.csv")

In [29]:
df_movies.head(5)

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Titulo_libros_prov
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",None
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",None
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",None
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",None
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",None


In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://chromedriver.chromium.org/downloads")

In [6]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [21]:
resultado = []
def busqueda(lib):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://www.goodreads.com/")
    sleep(1)
    driver.find_element("css selector","#sitesearch_field").click()
    driver.find_element("css selector","#sitesearch_field").send_keys(lib, Keys.ENTER)
    sleep(2)
    try:
        driver.implicitly_wait(2)
        driver.find_element("css selector","body > div:nth-child(3) > div > div > div.modal__close > button > img").click()
        try:
            sleep(2)
            driver.implicitly_wait(2)
            try:
                driver.find_element("xpath","/html/body/div[2]/div[3]/div[1]/div[2]/div[2]/table/tbody/tr[1]/td[2]/a/span").click()
                driver.implicitly_wait(2)
                #Autor
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__contributor > h3 > div > span:nth-child(1) > a > span.ContributorLink__name").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookAuthors > span:nth-child(2) > div > a").text)
                #Nota media del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(1) > div").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > span:nth-child(2)").text)
                #Numero de votantes
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(2) > div > span:nth-child(1)").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > a:nth-child(7)").text)     
                #Titulo del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageTitleSection > div.BookPageTitleSection__title > h1").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookTitle").text)
                """#Frase famosa
                sleep(1)
                try:
                    driver.find_element("css selector","#__next > div > main > div:nth-child(4) > div > div > a:nth-child(1) > div.DiscussionCard__middle > div.DiscussionCard__stats").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")
                except:
                    driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.rightContainer > div:nth-child(12) > div.bigBoxBody > div > a").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")"""
                
            except:
                driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > table > tbody > tr:nth-child(1) > td:nth-child(2) > a > span").click()
                driver.implicitly_wait(2)
                #Autor
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__contributor > h3 > div > span:nth-child(1) > a > span.ContributorLink__name").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookAuthors > span:nth-child(2) > div > a").text)
                #Nota media del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(1) > div").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > span:nth-child(2)").text)
                #Numero de votantes
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(2) > div > span:nth-child(1)").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > a:nth-child(7)").text)     
                #Titulo del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageTitleSection > div.BookPageTitleSection__title > h1").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookTitle").text)
                """#Frase famosa
                sleep(1)
                try:
                    driver.find_element("css selector","#__next > div > main > div:nth-child(4) > div > div > a:nth-child(1) > div.DiscussionCard__middle > div.DiscussionCard__stats").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")
                except:
                    driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.rightContainer > div:nth-child(12) > div.bigBoxBody > div > a").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")"""
                     
        except:
            resultado.append("No tiene autor")           
            resultado.append("No tiene nota media")
            resultado.append("No tiene cuento de votos")
            resultado.append("No tiene libro")
            return resultado
    except:
        try:
            sleep(2)
            driver.implicitly_wait(10)
            try:
                driver.find_element("xpath","/html/body/div[2]/div[3]/div[1]/div[2]/div[2]/table/tbody/tr[1]/td[2]/a/span").click()
                sleep(2)
                driver.implicitly_wait(4)
                #Autor
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__contributor > h3 > div > span:nth-child(1) > a > span.ContributorLink__name").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookAuthors > span:nth-child(2) > div > a").text)
                #Nota media del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(1) > div").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > span:nth-child(2)").text)
                #Numero de votantes
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(2) > div > span:nth-child(1)").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > a:nth-child(7)").text)                
                #Titulo del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageTitleSection > div.BookPageTitleSection__title > h1").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookTitle").text)
                """#Frase famosa
                sleep(1)
                try:
                    driver.find_element("css selector","#__next > div > main > div:nth-child(4) > div > div > a:nth-child(1) > div.DiscussionCard__middle > div.DiscussionCard__stats").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")
                except:
                    driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.rightContainer > div:nth-child(12) > div.bigBoxBody > div > a").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")"""
                
            except:
                driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > table > tbody > tr:nth-child(1) > td:nth-child(2) > a > span").click()
                sleep(2)
                driver.implicitly_wait(4)
                #Autor
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__contributor > h3 > div > span:nth-child(1) > a > span.ContributorLink__name").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookAuthors > span:nth-child(2) > div > a").text)
                #Nota media del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(1) > div").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > span:nth-child(2)").text)
                #Numero de votantes
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageMetadataSection > div.BookPageMetadataSection__ratingStats > a > div:nth-child(2) > div > span:nth-child(1)").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookMeta > a:nth-child(7)").text)
                #Titulo del libro
                try:
                    resultado.append(driver.find_element("css selector","#__next > div > main > div.BookPage__gridContainer > div.BookPage__rightColumn > div.BookPage__mainContent > div.BookPageTitleSection > div.BookPageTitleSection__title > h1").text)
                except:
                    resultado.append(driver.find_element("css selector","#bookTitle").text)
                """"#Frase famosa
                sleep(1)
                try:
                    driver.find_element("css selector","#__next > div > main > div:nth-child(4) > div > div > a:nth-child(1) > div.DiscussionCard__middle > div.DiscussionCard__stats").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")
                except:
                    driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.rightContainer > div:nth-child(12) > div.bigBoxBody > div > a").click()
                    sleep(2)
                    driver.implicitly_wait(3)
                    try:
                        resultado.append(driver.find_element("css selector","body > div.content > div.mainContentContainer > div.mainContent > div.mainContentFloat > div.leftContainer > div:nth-child(11) > div > div.quoteDetails > div.quoteText").text)
                        return resultado
                    except:
                        return resultado.append("No tiene frase famosa")"""
                
        except:
            resultado.append("No tiene autor")
            resultado.append("No tiene nota media")
            resultado.append("No tiene cuento de votos")
            resultado.append("No tiene libro")
            return resultado


In [22]:
libro = []
for i in df_movies["Series_Title"]:
    busqueda(i)
    print(resultado)
    libro.append(resultado)
    resultado = []

['Stephen King', '4.48', '2,430 ratings', 'The Shawshank Redemption: Different Seasons']
['Mario Puzo', '4.38', '390,111 ratings', 'The Godfather']
['Frank Miller', '4.26', '179,633 ratings', 'Batman: The Dark Knight Returns']
['Elizabeth Tubbs', '5.00', '1 rating', 'The Godfather Part II: Screenplay']
['Reginald Rose', '4.07', '29,209 ratings', 'Twelve Angry Men']
['Jude Fisher', '4.62', '22,171 ratings', 'The Lord of the Rings: The Return of the King: Visual Companion']
['Quentin Tarantino', '4.20', '6,006 ratings', 'Pulp Fiction: A Quentin Tarantino Screenplay']
['Thomas Keneally', '4.34', '155,888 ratings', "Schindler's List"]
['Bianca Scardoni', '3.98', '11,740 ratings', 'Inception']
['Chuck Palahniuk', '4.19', '553,401 ratings', 'Fight Club']
['Jude Fisher', '4.65', '31,104 ratings', 'The Lord of the Rings: The Fellowship of the Ring: Visual Companion']
['Winston Groom', '4.04', '65,412 ratings', 'Forrest Gump']
['Sergio Della Sala', '3.00', '4 ratings', 'Le neuroscienze a scuola

['Randall Wallace', '4.21', '12,971 ratings', 'Braveheart']
['Quentin Tarantino', '4.37', '1,260 ratings', 'Reservoir Dogs']
['Matthew Modine', '4.03', '213 ratings', 'Full Metal Jacket Diary']
['IWikipedia', '0.00', '0 ratings', 'Lauryeaty Moskovskogo Kinofestivalya: Tegeran-43, Voi Na I Mir, IDI I Smotri, Uolters, Dzhuli, Mimino, Vosem S Polovinoi']
['Ruby Dixon', '3.80', '48,397 ratings', 'Barbarian Alien']
['Peter Shaffer', '4.15', '18,115 ratings', 'Amadeus']
['Roben Farzad', '3.78', '701 ratings', 'Hotel Scarface: Where Cocaine Cowboys Partied and Plotted to Control Miami']
['Ryder Windham', '4.11', '412 ratings', 'Star Wars: Episode VI: Return of the Jedi']
['Lothar-Günther Buchheim', '4.25', '4,776 ratings', 'Das Boot']
['Hilary Winston', '3.39', '4,136 ratings', "My Boyfriend Wrote a Book About Me: And Other Stories I Shouldn't Share with Acquaintances, Coworkers, Taxi drivers, Assistants, Job Interviewers, Bikini Waxers, and Ex/Current/Future Boyfriends but Have"]
['S.D. Simp

['Tarryn Fisher', '3.99', '14,928 ratings', 'Bad Mommy']
['Corine Hartman', '3.54', '410 ratings', 'Bloedlijn']
['Mildred D. Taylor', '3.87', '120,619 ratings', 'Roll of Thunder, Hear My Cry']
['Emma Donoghue', '4.07', '775,531 ratings', 'Room']
['Sofía Esposito', '3.38', '8 ratings', 'Relatos eróticos para adultos sexo gratis en español: Relatos salvajes, relatos cortos, cuentos eróticos, lésbico, gay, humor, terror, crimen, historias de sexo duro y salvaje']
['Cassandra Clare', '4.23', '498,989 ratings', 'City of Lost Souls']
['Mehmet Eroğlu', '4.01', '98 ratings', 'Belleğin Kış Uykusu']
['P.K. Balakrishnan', '4.03', '1,832 ratings', 'ഇനി ഞാൻ ഉറങ്ങട്ടെ | Ini Njan Urangatte']
['Bob Kat', '3.93', '90 ratings', 'OMG (Oh My God)']
['Matt Zoller Seitz', '4.46', '741 ratings', 'The Wes Anderson Collection: The Grand Budapest Hotel']
['Gillian Flynn', '4.12', '2,811,132 ratings', 'Gone Girl']
['Mamoru Hosoda', '4.25', '693 ratings', 'おおかみこどもの雨と雪 1 [Ōkami kodomo no Ame to Yuki Vol. 1]']
['Bo

['No tiene autor', 'No tiene nota media', 'No tiene cuento de votos', 'No tiene libro']
['No tiene autor', 'No tiene nota media', 'No tiene cuento de votos', 'No tiene libro']
['Jerome Lawrence', '3.94', '22,809 ratings', 'Inherit the Wind']
['Marcel Moussy', '4.50', '2 ratings', 'Les Quatre Cents Coups - Scénario du film']
['Lew Wallace', '4.03', '34,980 ratings', 'Ben-Hur: A Tale of the Christ']
['No tiene autor', 'No tiene nota media', 'No tiene cuento de votos', 'No tiene libro']
['Source Wikipedia', '0.00', '0 ratings', "Film Diretti Da Federico Fellini: La Dolce Vita, 81/2, Amarcord, Giulietta Degli Spiriti, I Vitelloni, Prova D'Orchestra, Fellini Satyricon, Il Casanova Di Federico Fellini, La Strada, Le Notti Di Cabiria, Roma, La Citta Delle Donne"]
['dewakensyozouko', '0.00', '0 ratings', 'kumonosumura jouhenn']
['Pierre Boulle', '3.97', '9,090 ratings', 'The Bridge On The River Kwai']
['Mike McCoy', '4.80', '5 ratings', 'On the Waterfront']
['Georges Arnaud', '3.61', '382 rati

['Banco Central do Brasil', '3.00', '1 rating', 'Cartilha de Câmbio Banco Central do Brasil']
['Ted Hughes', '3.90', '10,639 ratings', 'The Iron Giant']
['Taylor Lee', '4.38', '16 ratings', "Knockin' On Heaven's Door (The Olive or Twist Saga Book 2)"]
['Billy Bob Thornton', '4.47', '79 ratings', 'Sling Blade']
['John Carreyrou', '4.41', '236,385 ratings', 'Bad Blood: Secrets and Lies in a Silicon Valley Startup']
['Elizabeth Hand', '3.70', '580 ratings', 'Twelve Monkeys']
['Tadashi Nagase', '0.00', '0 ratings', 'Kōkaku Kidōtai Mekanikaru Kaiseki Dokuhon']
['Shea Ernshaw', '3.94', '9,756 ratings', 'Long Live the Pumpkin Queen: Tim Burton’s The Nightmare Before Christmas']
['Robb Pearlman', '3.75', '348 ratings', "Groundhog's Day Off"]
['Cora Reilly', '3.86', '82,820 ratings', 'Bound by Honor']
['Giovanni Arpino', '3.06', '794 ratings', 'Scent of a Woman']
['Walt Disney Company', '4.37', '9,909 ratings', 'Walt Disney Aladdin']
['Fredrik Logevall', '4.54', '1,788 ratings', 'JFK: Coming of

...

['Haruki Ueno', '4.09', '1,006 ratings', 'Big Hero 6, Vol. 1']
['Elizabeth Young', '3.73', '7,070 ratings', 'A Promising Man']
['CHHATRA PAL VERMA', '0.00', '0 ratings', 'ENGLISH VINGLISH MATH-BATH']
['Tatsuo Hori', '3.95', '20 ratings', '風立ちぬ・美しい村[Kaze Tachinu Utsukushii Mura]']
['Drew Daywalt', '3.69', '184 ratings', 'Toy Story 4: Forky in Craft Buddy Day']
['Giuseppe Tornatore', '3.80', '159 ratings', 'La migliore offerta']
['Wes Anderson', '4.40', '238 ratings', 'Moonrise Kingdom']
['Cressida Cowell', '4.09', '16,909 ratings', 'How to Be a Pirate']
['Michael Lewis', '4.29', '152,272 ratings', 'The Big Short: Inside the Doomsday Machine']
['Kanae Minato', '4.06', '26,703 ratings', '告白 [Kokuhaku]']
['No tiene autor', 'No tiene nota media', 'No tiene cuento de votos', 'No tiene libro']
['Stieg Larsson', '4.16', '2,994,497 ratings', 'The Girl with the Dragon Tattoo']
['Richard Phillips', '3.94', '4,034 ratings', "A Captain's Duty: Somali Pirates, Navy SEALs, and Dangerous Days at Sea"]

['Tom Wolfe', '4.22', '50,478 ratings', 'The Right Stuff']
['Asad a.k.a. ‘The Literary King of Comedy’', '0.00', '0 ratings', 'Crackheads, Can’t Quit Tha Fix: Episode 2']
['William Kotzwinkle', '4.18', '10,934 ratings', 'E.T. the Extra-Terrestrial in His Adventure on Earth']
['Avery Corman', '3.85', '2,206 ratings', 'Kramer vs. Kramer']
['Randy Alcorn', '4.37', '372 ratings', '50 Days of Heaven: Reflections That Bring Eternity to Light']
['Forrest Carter', '4.07', '683 ratings', 'The Outlaw Josey Wales']
['the man who would be king', '2.00', '3 ratings', 'مذكرات سفير في مملكة الدجال']
['Judd Apatow', '3.65', '11,895 ratings', 'Sick in the Head: Conversations About Life and Comedy']
['Source Wikipedia', '0.00', '0 ratings', "Adaptation D'Une Uvre Litteraire de Science-Fiction Au Cinema: 2001, L'Odyssee de L'Espace, Les Maitres Du Temps, La Planete Sauvage, Les Fils de L'Ho"]
['Frederick Forsyth', '4.26', '130,062 ratings', 'The Day of the Jackal']
['Danielle Girard', '4.06', '11,781 rat

['J.K. Rowling', '4.57', '3,303,695 ratings', 'Harry Potter and the Goblet of Fire']
['A.J. Quinnell', '4.26', '3,788 ratings', 'Man on Fire']
['Neil Gaiman', '4.10', '613,089 ratings', 'Coraline']
['Helen DeWitt', '4.15', '7,758 ratings', 'The Last Samurai']
['No tiene autor', 'No tiene nota media', 'No tiene cuento de votos', 'No tiene libro']
['Wolfgang Becker', '3.57', '28 ratings', 'Good Bye, Lenin']
['Barbara Ehrenreich', '3.64', '187,590 ratings', 'Nickel and Dimed: On (Not) Getting by in America']
['Tish Rabe', '3.90', '68 ratings', 'I Am Sam-I-Am']
['Robert Sabuda', '4.34', '26,046 ratings', "Alice's Adventures in Wonderland: A Pop-Up Adaptation"]
['Mark Bowden', '4.29', '60,984 ratings', 'Black Hawk Down: A Story of Modern War']
['Max Allan Collins', '3.88', '2,661 ratings', 'Road to Perdition']
['Kelly Oram', '4.00', '14,029 ratings', 'Das Avery Shaw Experiment']
['Melvin Burgess', '3.79', '1,695 ratings', 'Billy Elliot']
['John Cameron Mitchell', '4.25', '902 ratings', 'Hed

['Victoria Aveyard', '4.02', '866,292 ratings', 'Red Queen']
['Marguerite Henry', '4.07', '3,351 ratings', 'Mustang: Wild Spirit of the West']
['Jacob Johnston', '4.30', '70 ratings', 'The Art of Guardians of the Galaxy Vol. 2']
['Jan Kerouac', '3.73', '327 ratings', 'Baby Driver: A Story About Myself']
['Mel Sherratt', '4.26', '2,239 ratings', 'Only the Brave']
['Giles Whittell', '3.83', '3,693 ratings', 'Bridge of Spies: A True Story of the Cold War']
['Matthew Bogart', '4.18', '117 ratings', 'Incredible Doom: Volume 2']
['Kalikolehua Hurley', '4.38', '777 ratings', 'Moana: The Mighty Maui Makes a Friend']
['Harry Hunsicker', '4.11', '418 ratings', 'Texas Sicario']
['Kristen Ashley', '4.16', '33,734 ratings', 'Creed']
['Boris Akunin', '4.05', '7,446 ratings', 'Левиафан [Leviafan]']
['S.E. Jakes', '3.96', '5,450 ratings', 'Catch a Ghost']
['Martin Sixsmith', '3.71', '16,526 ratings', 'The Lost Child of Philomena Lee: A Mother, Her Son and a 50 Year Search']
['Alexander C. Irvine', '4.

['Ernest Thompson', '4.10', '555 ratings', 'On Golden Pond']
['George Miller', '3.73', '129 ratings', 'Mad Max: Fury Road: Mad Max #2']
['Erin Hunter', '4.31', '93,392 ratings', 'Into the Wild']
['Katharine Turner', '3.59', '58 ratings', 'The Muppets: The Movie Junior Novel']
['J. Campbell Bruce', '3.61', '786 ratings', 'Escape from Alcatraz']
['Richard Adams', '4.07', '463,886 ratings', 'Watership Down']
['Billy Hayes', '4.04', '3,910 ratings', 'Midnight Express']
['Steven Spielberg', '4.11', '12,910 ratings', 'Close Encounters of the Third Kind']
['Raymond Chandler', '4.19', '39,529 ratings', 'The Long Goodbye']
['Henri-Clément Sanson', '3.00', '2 ratings', 'Giù la testa. Memorie del boia della rivoluzione']
['Rebecca Joyce', '4.02', '372 ratings', "Kelly's Heroes"]
['Rudyard Kipling', '3.91', '116,360 ratings', 'The Jungle Book']
['Todd Borg', '4.05', '429 ratings', 'Tahoe Blowup']
['Sherrilyn Kenyon', '3.91', '1,220 ratings', "A Hard Day's Night Searcher"]
['Truman Capote', '3.86',

In [ ]:
df_movies["partial_ratio"] = df_movies.apply(lambda x: fuzz.partial_ratio(x['Series_Title'], x['Genre']), axis=1)


In [156]:
lista_pochita = []
for (a,b) in zip(scrap_limpio[3],list(df_movies["Series_Title"])):
    porcentaje = fuzz.ratio(a,b)
    if porcentaje > 45:
        porcentaje = True
        lista_pochita.append(porcentaje)
    else:
        porcentaje = False
        lista_pochita.append(porcentaje)


In [157]:
lista_pochita

[True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 

In [162]:
counter= 0
lista_pochita = []
for (a,b) in zip(scrap_limpio[3],list(df_movies["Series_Title"])):
    porcentaje = fuzz.ratio(a,b)
    if porcentaje > 45:
        lista_pochita.append(counter)
    counter+=1

In [163]:
len(lista_pochita)

656

In [164]:
for n in lista_pochita:
    print (f"{scrap_limpio[3][n]}--------------{df_movies['Series_Title'][n]}")

The Shawshank Redemption: Different Seasons--------------The Shawshank Redemption
The Godfather--------------The Godfather
Batman: The Dark Knight Returns--------------The Dark Knight
The Godfather Part II: Screenplay--------------The Godfather: Part II
Twelve Angry Men--------------12 Angry Men
The Lord of the Rings: The Return of the King: Visual Companion--------------The Lord of the Rings: The Return of the King
Schindler's List--------------Schindler's List
Inception--------------Inception
Fight Club--------------Fight Club
The Lord of the Rings: The Fellowship of the Ring: Visual Companion--------------The Lord of the Rings: The Fellowship of the Ring
Forrest Gump--------------Forrest Gump
Le neuroscienze a scuola: Il buono, il brutto e il cattivo--------------Il buono, il brutto, il cattivo
The Lord of the Rings: The Two Towers (The Lord of the Rings, #2)--------------The Lord of the Rings: The Two Towers
Made Men: The Story of Goodfellas--------------Goodfellas
The Art of Star 

In [116]:
for (a,b) in zip(scrap_limpio[3],list(df_movies["Series_Title"])):
    print (f"{a} ----------------{b}")

P ----------------P
u ----------------u
l ----------------l
p ----------------p
  ---------------- 
F ----------------F
i ----------------i
c ----------------c
t ----------------t
i ----------------i
o ----------------o
n ----------------n
T ----------------T
h ----------------h
e ----------------e
  ---------------- 
D ----------------M
i ----------------a
v ----------------t
i ----------------r
n ----------------i
e ----------------x
M ----------------G
a ----------------o
d ----------------o
e ----------------d
  ----------------f
M ----------------e
e ----------------l
n ----------------l
: ----------------a
  ----------------s
N ----------------G
o ----------------i
  ----------------s
t ----------------a
i ----------------e
e ----------------n
n ----------------g
e ----------------c
  ----------------h
l ----------------u
i ----------------n
b ----------------g
N ----------------S
o ----------------o
  ----------------o
t ----------------r
i ----------------a
e ----------------r


In [ ]:
for i in df_movies["partial_ratio"]:
    if i >= 80:
        df_movies["Tiene_libro"] = True
    else:
        df_movies["Tiene_libro"] = False

In [61]:
libro2

[['Stephen King',
  '4.48',
  '2,430 ratings',
  'The Shawshank Redemption: Different Seasons'],
 ['Mario Puzo', '4.38', '390,111 ratings', 'The Godfather'],
 ['Frank Miller',
  '4.26',
  '179,633 ratings',
  'Batman: The Dark Knight Returns'],
 ['Elizabeth Tubbs', '5.00', '1 rating', 'The Godfather Part II: Screenplay'],
 ['Reginald Rose', '4.07', '29,209 ratings', 'Twelve Angry Men'],
 ['Jude Fisher',
  '4.62',
  '22,171 ratings',
  'The Lord of the Rings: The Return of the King: Visual Companion'],
 ['Quentin Tarantino',
  '4.20',
  '6,006 ratings',
  'Pulp Fiction: A Quentin Tarantino Screenplay'],
 ['Thomas Keneally', '4.34', '155,888 ratings', "Schindler's List"],
 ['Bianca Scardoni', '3.98', '11,740 ratings', 'Inception'],
 ['Chuck Palahniuk', '4.19', '553,401 ratings', 'Fight Club'],
 ['Jude Fisher',
  '4.65',
  '31,104 ratings',
  'The Lord of the Rings: The Fellowship of the Ring: Visual Companion'],
 ['Winston Groom', '4.04', '65,412 ratings', 'Forrest Gump'],
 ['Sergio Dell

SyntaxError: invalid syntax (1436873230.py, line 1)

In [94]:
def listas_libros(libros):
    lista_libros=[]
    for n in range(4):
        lista = [libro[n] for libro in libros]
        lista_libros.append(lista)
    return lista_libros

In [95]:
scrap_limpio = listas_libros(libro2)

In [96]:
scrap_limpio[0]

['Stephen King',
 'Mario Puzo',
 'Frank Miller',
 'Elizabeth Tubbs',
 'Reginald Rose',
 'Jude Fisher',
 'Quentin Tarantino',
 'Thomas Keneally',
 'Bianca Scardoni',
 'Chuck Palahniuk',
 'Jude Fisher',
 'Winston Groom',
 'Sergio Della Sala',
 'Brian Sibley',
 'Gregg Braden',
 'Glenn Kenny',
 'Deborah Call',
 'Ken Kesey',
 'Ron Chernow',
 'No tiene autor',
 'No tiene autor',
 'Michio Kaku',
 'Paulo Lins',
 'Hayao Miyazaki',
 'Jacqueline Kehl',
 'Stephen King',
 'Roberto Benigni',
 'S.L. Jennings',
 'Thomas Harris',
 'Timothy Zahn',
 'Peter Watts',
 'Akira Kurosawa',
 'Bob Welch',
 'Brian Azzarello',
 'Catherine Coulter',
 'Philippe Pozzo di Borgo',
 'Christopher Priest',
 'J.A. Templeton',
 'Władysław Szpilman',
 'Ruby Dixon',
 'Ibram X. Kendi',
 'Maurice Broaddus',
 'Leo Tolstoy',
 'Justine Korman Fontes',
 'Randall Frakes',
 'Franco Cristaldi',
 'Akiyuki Nosaka',
 'George Gipe',
 'Lori Austin',
 'Bret Easton Ellis',
 'Fiona Valpy',
 'Michael R. Beschloss',
 'Emma Scott',
 'Lewis Trondh